In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_log_error
from sklearn.model_selection import cross_validate, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.base import BaseEstimator, RegressorMixin
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('data/train.csv')
df = df.drop(columns='id')

test_df = pd.read_csv('data/test.csv')
test_df = test_df.drop(columns='id')
display(df.head())
display(test_df.head())

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


# Phase 1: Nonlog Features

In [3]:
X = df.drop(labels='Premium Amount', axis=1)
Y = df['Premium Amount']

In [ ]:
# Hist gb
hist_X = X.copy()
hist_X['Policy Year'] = pd.to_datetime(hist_X['Policy Start Date']).dt.year
hist_X = hist_X.drop('Policy Start Date', axis=1)

hist_X= hist_X.fillna('None')

k = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
oof = np.zeros(len(hist_X))

for index, (train_index, test_index) in enumerate(k.split(hist_X)):
    hist_model = CatBoostRegressor(
            iterations=1000,
            learning_rate=0.1,
            depth=6,
            eval_metric="RMSE",
            random_seed=42,
            verbose=200,
            task_type='CPU',
            l2_leaf_reg =  0.7,
        )

    hist_model.fit(hist_X.iloc[train_index],
                   Y.iloc[train_index],
                   early_stopping_rounds=300,
                   cat_features = hist_X.columns.values)
    predictions = hist_model.predict(hist_X.iloc[test_index])
    oof[test_index] = predictions

In [44]:
root_mean_squared_log_error(Y, oof)

1.13139840740729

# Part 2: New Model on the Nonlogs

In [20]:
nonlog_X = hist_X.copy()
nonlog_X['nonlog'] = oof

hist_model = HistGradientBoostingRegressor(max_iter=100, categorical_features=categorical_indices)
res = cross_validate(estimator=hist_model, X=nonlog_X, y=np.log1p(Y), scoring='neg_root_mean_squared_error')
res['test_score'].mean()

-1.0458457209887448

In [34]:
# Try again but delete other shit

def extract_policy_year(df):
    df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'])
    df['Year'] = df['Policy Start Date'].dt.year
    df['Day'] = df['Policy Start Date'].dt.day
    df['Month'] = df['Policy Start Date'].dt.month
    df['Month_name'] = df['Policy Start Date'].dt.month_name()
    df['Day_of_week'] = df['Policy Start Date'].dt.day_name()
    df['Week'] = df['Policy Start Date'].dt.isocalendar().week
    df['Year_sin'] = np.sin(2 * np.pi * df['Year'])
    df['Year_cos'] = np.cos(2 * np.pi * df['Year'])
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12) 
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Day_sin'] = np.sin(2 * np.pi * df['Day'] / 31)  
    df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 31)
    df['Group'] = (df['Year'] - 2020) * 48 + df['Month'] * 4 + df['Day'] // 7

    df.drop('Policy Start Date', axis=1, inplace=True)
    return df

def modify_numerical(df):
    df = df.copy()
    df = df.drop(columns=['Age'])
    df['Number of Dependents'] = df['Number of Dependents'].isna().astype(int)
    df['Vehicle Age'] = df['Vehicle Age'].isna().astype(int)
    df['Insurance Duration'] = df['Insurance Duration'].isna().astype(int)
    return df

def modify_categorical(df):
    df = df.copy()
    df = df.drop(columns=['Gender', 'Education Level', 'Location', 'Policy Type', \
                          'Smoking Status', 'Exercise Frequency', 'Property Type'])
    df['Marital Status'] = df['Marital Status'].isna().astype(int)
    df['Occupation'] = df['Occupation'].isna().astype(int)
    df['Customer Feedback'] = df['Customer Feedback'].isna().astype(int)
    return df

def combine(df):
    df = extract_policy_year(df)
    df = modify_numerical(df)
    df = modify_categorical(df)
    return df

new_X = extract_policy_year(X)
new_X['nonlog'] = oof

In [35]:
categorical_indices = [new_X.columns.get_loc(col) for col in new_X.select_dtypes(include=['object']).columns]

hist_model = HistGradientBoostingRegressor(max_iter=100, categorical_features=categorical_indices)
res = cross_validate(estimator=hist_model, X=new_X, y=np.log1p(Y), scoring='neg_root_mean_squared_error')
res['test_score'].mean()

-1.0459229691340362